Encuesta de Gasto e Ingresos de los Hogares - Engih 2016 - 2017


La Encuesta de Gasto e Ingresos de los Hogares (ENGIH) 2016-2017 en Uruguay es un estudio fundamental realizado por el Instituto Nacional de Estadística (INE), cuyo propósito principal es obtener información detallada sobre el gasto, ingreso y condiciones de vida de los hogares uruguayos. A continuación te detallo sus principales características y objetivos:

Características de la ENGIH 2016-2017:
Periodicidad: Es una encuesta de carácter periódico, que generalmente se realiza cada 10 años aproximadamente, con el fin de actualizar la canasta de consumo que se utiliza para el cálculo del Índice de Precios al Consumo (IPC).

    Cobertura: La ENGIH abarca todo el territorio nacional, incluyendo tanto zonas urbanas como rurales. Se encuesta una muestra representativa de hogares, permitiendo la inferencia de resultados a nivel nacional, por regiones y por distintos niveles socioeconómicos.

    Método de recolección: La encuesta se realiza mediante entrevistas directas en los hogares, con el uso de un diario de gastos en el cual las familias registran todos sus gastos cotidianos por un período determinado.

Contenido: La encuesta recopila información sobre:

    Ingresos del hogar: Ingresos laborales y no laborales, pensiones, transferencias, entre otros.
    Gastos del hogar: En bienes y servicios de consumo, gastos en salud, educación, transporte, alimentos, vivienda, etc.
    Condiciones de vida: Características del hogar, tenencia de bienes durables, acceso a servicios básicos, entre otros.

Duración del relevamiento: Se realizó a lo largo de un año, desde diciembre de 2016 hasta diciembre de 2017, con el fin de captar variaciones estacionales en los patrones de gasto e ingreso.



Objetivos de la ENGIH 2016-2017:

1.	Actualización de la canasta de consumo: Uno de los principales objetivos es actualizar la canasta de bienes y servicios que se utiliza para calcular el Índice de Precios al Consumo (IPC), el cual mide la evolución de los precios de bienes y servicios consumidos por los hogares uruguayos.

2.	Medición de la distribución del ingreso: La ENGIH permite estimar la distribución del ingreso en los hogares, lo que contribuye al análisis de la pobreza, la desigualdad y la equidad en Uruguay.

3.	Conocer patrones de consumo: Ayuda a entender los patrones de gasto de los hogares uruguayos, identificando en qué productos y servicios destinan sus recursos, lo que es esencial para la formulación de políticas públicas y decisiones de mercado.

4.	Análisis de condiciones de vida: La encuesta permite estudiar las condiciones de vida de los hogares, como la calidad de la vivienda, el acceso a servicios básicos, la tenencia de bienes, etc., información clave para el diseño de políticas sociales y económicas.

5.	Evaluación de políticas públicas: Provee una base de datos robusta para evaluar el impacto de diversas políticas públicas en la economía de los hogares, especialmente en relación con la pobreza, el empleo, y las transferencias de ingresos.

6.	Estimación de la demanda de bienes y servicios: Aporta insumos cruciales para la planificación y el diseño de políticas económicas y sociales, proporcionando datos sobre la estructura de la demanda interna de bienes y servicios.

La ENGIH 2016-2017 es una herramienta vital para entender las dinámicas económicas y sociales de los hogares en Uruguay, y sus resultados son utilizados para la toma de decisiones en múltiples áreas, desde el cálculo de indicadores macroeconómicos hasta la formulación de políticas sociales.

En éste trabajo vamos a analizar

1.	Calorías totales
2.	Distribución porcentual de macronutrientes
4.	Ultra procesados

Todo con los filtros de
1.	Región, 
2.	quintiles y 
3.	hogares con menores de 5 años y sin niños menores de 5 años


Lo primero que hacemos es quedarnos con la información de la Tabla de Gastos Engih 2016 que queremos. Nos quedamos con el consumo con destino al Hogar.

In [18]:
import pandas as pd
import numpy as np
import openpyxl

# Cargar archivo de gastos 
ruta_gastos = '../data/ENGIH 2016 Base de Datos Gastos.xlsx' 
df_gastos = pd.read_excel(ruta_gastos)

# Crear una nueva columna `gasto_hogar` basada en las condiciones (DESTINOCODIGO 1 o 5)
df_gastos['gasto_hogar'] = np.where((df_gastos['DESTINOCODIGO'] == 1) | (df_gastos['DESTINOCODIGO'] == 5), 1, 0)

# Filtrar por gasto_hogar igual a 1
df_gastos_filtrado = df_gastos[df_gastos['gasto_hogar'] == 1]



In [24]:
df_gastos_filtrado['CALORIAS'].describe()

count    494780.000000
mean        596.001638
std        1880.787938
min           0.000000
25%           0.000000
50%           0.000000
75%         611.493250
max      360915.000000
Name: CALORIAS, dtype: float64

A partir de acá macheamos las tablas asociadas a la Tabla de Gastos Engih 2016

In [19]:
# Cargar la tabla de composición química
ruta_composicion = '../data/TC_MACH.xlsx' 
df_composicion = pd.read_excel(ruta_composicion)

# Hacer el match por 'ARTICULOCODIGO'
df_merged = pd.merge(df_gastos_filtrado, df_composicion, on='ARTICULOCODIGO', how='left')

# Hacer el match con otras tablas
ruta_ninos_hogar = '../data/Niños hogar.xlsx' 
df_ninos_hogar = pd.read_excel(ruta_ninos_hogar)

df_merged = pd.merge(df_merged, df_ninos_hogar, on='NUMERO', how='left')

# Cargar otras tablas y hacer merge con CCIF_2016
ruta_grupos = '../data/Grupos 2006 y ultra msp.xlsx' 
df_grupos = pd.read_excel(ruta_grupos)

# Eliminar la columna 'CCIF_2016_y'
df_merged = df_merged.drop(columns=['CCIF_2016_y'])

# Renombrar 'CCIF_2016_x' a 'CCIF_2016'
df_merged = df_merged.rename(columns={'CCIF_2016_x': 'CCIF_2016'})

df_merged = pd.merge(df_merged, df_grupos, on='CCIF_2016', how='left')

# Cargar variables ingresos y hacer merge con NUMERO
ruta_ingresos = '../data/variables ingresos.xlsx' 
df_ingresos = pd.read_excel(ruta_ingresos)

df_merged = pd.merge(df_merged, df_ingresos, on='NUMERO', how='left')



Convertimos las cantidades a cantidades consumidas ejecutando lo que se llama Factor de Corrección

In [20]:
# Recalcular CANTIDADCONVERTIDA y calorías finales
df_merged['CANTIDADCONVERTIDA'] = df_merged['CANTIDADCONVERTIDA'] / df_merged['FC']
df_merged['Calorias_final'] = df_merged['CANTIDADCONVERTIDA'] * df_merged['Calorias_2016'] / 100

# Calcular gr_mes basado en la cantidad convertida
df_merged['gr_mes'] = df_merged['CANTIDADCONVERTIDA'] * 4.3

In [21]:
# Crear la columna de Niños_hogar basado en la suma de Niños_hogar_sum
df_merged['Niños_hogar'] = np.where(df_merged['Niños_hogar_sum'] >= 1, 1, 0)

# Asignar etiquetas a la nueva columna
df_merged['Niños_hogar'] = df_merged['Niños_hogar'].map({0: "Hogar sin niños", 1: "Hogar con niños"})


In [22]:
# Cálculo de gramos por día
df_merged['gr_dia'] = df_merged['CANTIDADCONVERTIDA'] * 52.179 / 366

# Agregar por 'NUMERO' para calcular gramos por hogar
df_gr_hogar = df_merged.groupby('NUMERO', as_index=False)['gr_dia'].sum()
df_gr_hogar.rename(columns={'gr_dia': 'gr_hogar'}, inplace=True)

# Calcular gramos per cápita
df_gr_hogar['gr_percapita'] = df_gr_hogar['gr_hogar'] / df_merged['nper_hogar']

# Calcular calorías por artículo por día
df_merged['Cal_articulo_dia'] = df_merged['gr_dia'] * df_merged['Calorias_2016'] / 100

# Agregar por 'NUMERO' para calorías por hogar
df_cal_hogar = df_merged.groupby('NUMERO', as_index=False)['Cal_articulo_dia'].sum() #aca esta mal eso???
df_cal_hogar.rename(columns={'Cal_articulo_dia': 'Calorias_hogar'}, inplace=True)

# Calorías per cápita
df_cal_hogar['Cal_percapita'] = df_cal_hogar['Calorias_hogar'] / df_merged['nper_hogar']

# Primero, asegurarte de que ambos DataFrames tengan una columna 'NUMERO' para hacer el merge
df_merged = df_merged.merge(df_gr_hogar[['NUMERO', 'gr_percapita']], on='NUMERO', how='left')
df_merged = df_merged.merge(df_cal_hogar[['NUMERO', 'Cal_percapita']], on='NUMERO', how='left')

# Merge de gramos por hogar
df_merged = df_merged.merge(df_gr_hogar[['NUMERO', 'gr_hogar']], on='NUMERO', how='left')

# Merge de calorías por hogar
df_merged = df_merged.merge(df_cal_hogar[['NUMERO', 'Calorias_hogar']], on='NUMERO', how='left')



In [15]:
# Cálculo de proteínas en gramos y su energía en calorías
df_merged['gr_proteinas'] = df_merged['gr_dia'] * df_merged['Proteinas_2016'] / 100
df_merged['gr_proteinas'].fillna(0, inplace=True)

# Agregar proteínas por hogar
df_prot_hogar = df_merged.groupby('NUMERO', as_index=False)['gr_proteinas'].sum()
df_prot_hogar.rename(columns={'gr_proteinas': 'prot_hogar'}, inplace=True)
df_prot_hogar['cal_proteinas'] = df_prot_hogar['prot_hogar'] * 4

# Cálculo de grasas en gramos y su energía en calorías
df_merged['gr_grasas'] = df_merged['gr_dia'] * df_merged['Grasas_2016'] / 100
df_merged['gr_grasas'].fillna(0, inplace=True)

# Agregar grasas por hogar
df_grasas_hogar = df_merged.groupby('NUMERO', as_index=False)['gr_grasas'].sum()
df_grasas_hogar.rename(columns={'gr_grasas': 'grasas_hogar'}, inplace=True)
df_grasas_hogar['cal_grasas'] = df_grasas_hogar['grasas_hogar'] * 9

# Cálculo de hidratos en gramos y su energía en calorías
df_merged['gr_hidratos'] = df_merged['gr_dia'] * df_merged['Hidratos_2016'] / 100
df_merged['gr_hidratos'].fillna(0, inplace=True)

# Agregar hidratos por hogar
df_hidratos_hogar = df_merged.groupby('NUMERO', as_index=False)['gr_hidratos'].sum()
df_hidratos_hogar.rename(columns={'gr_hidratos': 'hidratos_hogar'}, inplace=True)
df_hidratos_hogar['cal_hidratos'] = df_hidratos_hogar['hidratos_hogar'] * 4

# Sumar las calorías de los macronutrientes
df_calorias_macro = pd.merge(df_prot_hogar, df_grasas_hogar, on='NUMERO')
df_calorias_macro = pd.merge(df_calorias_macro, df_hidratos_hogar, on='NUMERO')
df_calorias_macro['calorias_macro'] = df_calorias_macro['cal_proteinas'] + df_calorias_macro['cal_grasas'] + df_calorias_macro['cal_hidratos']

#Adecuación de los macronutrientes
df_calorias_macro['adec_proteinas'] = df_calorias_macro['cal_proteinas'] * 100 / df_calorias_macro['calorias_macro']
df_calorias_macro['adec_grasas'] = df_calorias_macro['cal_grasas'] * 100 / df_calorias_macro['calorias_macro']
df_calorias_macro['adec_hidratos'] = df_calorias_macro['cal_hidratos'] * 100 / df_calorias_macro['calorias_macro']

df_merged = df_merged.merge(df_calorias_macro[['NUMERO', 'cal_proteinas', 'cal_grasas', 'cal_hidratos',
                                               'adec_proteinas', 'adec_grasas', 'adec_hidratos']], on='NUMERO', how='left')


In [16]:
# Ultraprocesados 
df_merged['gr_ULT'] = np.where(df_merged['Ultra_mps'] == 1, df_merged['gr_dia'], 0)
df_merged['cal_ULT'] = np.where(df_merged['Ultra_mps'] == 1, df_merged['Cal_articulo_dia'], 0)

# Agregar gramos y calorías de ultraprocesados por hogar
df_aggregated_ultra = df_merged.groupby('NUMERO', as_index=False).agg({
    'gr_ULT': 'sum',
    'cal_ULT': 'sum'
})

# Obtener 'nper_hogar' por 'NUMERO' (asumiendo que es constante por 'NUMERO')
df_nper_hogar = df_merged[['NUMERO', 'nper_hogar']].drop_duplicates()

# Merge 'nper_hogar' en 'df_aggregated_ultra'
df_aggregated_ultra = pd.merge(df_aggregated_ultra, df_nper_hogar, on='NUMERO', how='left')

# Calcular per cápita
df_aggregated_ultra['gr_ultra_percapita'] = df_aggregated_ultra['gr_ULT'] / df_aggregated_ultra['nper_hogar']
df_aggregated_ultra['cal_ultra_percapita'] = df_aggregated_ultra['cal_ULT'] / df_aggregated_ultra['nper_hogar']



In [17]:
# Fusionar las columnas calculadas de ultraprocesados en 'df_merged_unique'
df_merged = pd.merge(
    df_merged,
    df_aggregated_ultra[['NUMERO', 'gr_ultra_percapita', 'cal_ultra_percapita']],
    on='NUMERO',
    how='left'
)

# Calcular la proporción de calorías ultraprocesadas en el DataFrame Final
df_merged['prop_ultra_percapita'] = (
    df_merged['cal_ultra_percapita'] * 100 / df_merged['Cal_percapita']
)

# Identificar duplicados basados en la columna 'NUMERO' y quedarse con solo uno
df_merged_unique = df_merged.drop_duplicates(subset='NUMERO', keep='first')



In [9]:
Columnas_remover = ['DOMANIO',  'DOMSEMANA', 'CCIF_2016', 'ARTICULOCODIGO', 'DONDECODIGO', 'COMOCODIGO',
'DESTINOCODIGO', 'GASTOTIPO', 'DIVISION', 'VALORCONT', 'VALORCONTM',
'CANTIDADCONVERTIDA', 'VL_IMPUTADO', 'VALORCONT_IMPUTADO', 'CALORIAS',
'gasto_hogar', 'Descripción_CCIF_2016', 'FC', 'Calorias_2016',
'Proteinas_2016', 'Grasas_2016', 'Hidratos_2016', 'Alimentos',
'GABA_2016', 'recomendados', 'grupos_2006', 'HT11', 'Calorias_final', 'gr_mes',
'Cal_articulo_dia']

df_merged_unique = df_merged_unique.drop(columns=Columnas_remover)
   
# Guardar el DataFrame resultante en un nuevo archivo Excel
df_merged_unique.to_excel('../data/Hogar_Nutrientes_Final.xlsx', index=False)

print("Archivo guardado con casos únicos por 'NUMERO'.")


Archivo guardado con casos únicos por 'NUMERO'.


In [3]:
#pip freeze > requirements.txt 

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

df = pd.read_excel('../data/Hogar_Nutrientes_Final.xlsx')

In [2]:
df['Cal_percapita'].describe()

count     6889.000000
mean      2694.194149
std       3004.809687
min          0.000000
25%        968.989459
50%       1811.097388
75%       3293.443835
max      91321.475522
Name: Cal_percapita, dtype: float64